In [2]:
import urllib2
homer = urllib2.urlopen('http://people.sc.fsu.edu/~jburkardt/datasets/sgb/homer.dat')

# Connected Components

The purpose of this assignment is to familiarize yourself with the handling of graph data structures. You will implement the algorithm for identifying the connected components of an undirected graph, implementing depth-first-search as a subroutine along the way.

You will use the [NetworkX](https://networkx.github.io/) Python package to represent and manipulate graphs. You should first familiarize yourself with its functionality by going through the brief [tutorial](http://networkx.github.io/documentation/networkx-1.9.1/tutorial/index.html). For this homework, you may only use the basic undirected graph methods listed [here](http://networkx.github.io/documentation/networkx-1.9.1/reference/classes.graph.html).

As a use case, we will work with a dataset recording the interactions between characters in Homer's *Iliad*.

The format of the data is straightforward. After some comment lines (beginning with \*), the file lists a codename for each character (i.e., node of the graph), followed by a description. The file then lists the groups of characters that interact in each chapter, from which you will form the edges. For instance, the first line has the form:

```1:CH,AG,ME,GS;AP,CH;HE,AC;AC,AG,CA;HE,AT;AT,AC;AT,OG;NE,AG,AC;CS,OD```

This means that CH,AG,ME,GS interacted, so there are edges for all pairs of these nodes. Groups of characters that interacted are separated by semicolons. The lines start with chapter information of the form `1:` or `&:`, which can be ignored for this problem.

First implement a function to read in the nodes from the input file. You may implement any auxiliary functions as needed, and are encouraged to use small functions with specific purposes to keep your code readable. Any function you implement should be clearly commented.

In [3]:
def read_nodes(gfile):
    """Reads in the nodes of the graph from the input file.
    
    Args:
        gfile: A handle for the file containing the graph data, starting at the top.
        
    Returns:
        A generator of the nodes in the graph, yielding a list of the form:
            ['CH', 'AG, 'ME', ...]
         """
    # TODO: implement function
    nodeList = []
    for line in homer:
    # Split the line by a whitespace
        temp = line.split(' ')[0]
        if temp == "\n":
            break
        if temp != "*":
    # Append the node just read in to nodeList
            nodeList.append(line.split(' ')[0])
    return nodeList
    

Next implement a function to read in the edges from the input file.

In [4]:
def read_edges(gfile):
    """Reads in the edges of the graph from the input file.
    
    Args:
        gfile: A handle for the file containing the graph data, starting at the top 
            of the edges section.
            
    Returns:
        A generator of the edges in the graph, yielding a list of pairs of the form:
            [('CH', 'AG'), ('AG', 'ME'), ...]
    """
    # TODO: implement function
    edgeList = []
    for line in homer:
    # Split the line by a ':'
        temp = line.split(':')
        if len(temp) > 1:
            temp = temp[1]
    # Strip the \n and white space
            temp = temp.rstrip()
            temp = temp.split(';')
            for str in temp:
                tempTwo = str.split(',')
    # Build the pairs without repeat
                tempTwo = [(x,y) for x in tempTwo for y in tempTwo if x>y]
                edgeList = list(set(edgeList)|set(tempTwo))             
    return edgeList

The following code should now correctly create the graph.

In [5]:
import networkx as nx
G = nx.Graph()
G.add_nodes_from(read_nodes(homer))
G.add_edges_from(read_edges(homer))
print G.nodes()

['G2', 'G1', 'G0', 'GS', 'GR', 'GL', '3Z', '3Y', '3X', '3S', '3R', '3Q', '3P', '3W', '3V', '3U', '3T', '3K', '3J', '3I', '3H', '3O', '3N', '3M', '3L', '3C', '3B', '3A', '3G', '3F', '3E', '3D', 'Z0', 'Z1', 'ZE', 'ZF', 'ME', 'MG', '38', 'MC', '33', '32', '31', '30', '37', '36', '35', '34', 'MU', 'MT', 'SE', 'MR', 'FY', 'S1', 'S0', 'FD', '9I', '9H', '9K', '9J', '9M', '9L', '9O', '9N', '9A', '9C', '9B', '9E', '9D', '9G', '9F', '9Y', '9X', '9Z', '9Q', '9P', '9S', '9R', '9U', '9T', '9W', '9V', '99', '98', '91', '90', '93', '92', '95', '94', '97', '96', 'F0', 'F1', 'F2', 'SP', 'ST', 'SI', 'SL', 'SA', 'SF', '0U', 'LE', 'LA', 'LT', 'LY', '7K', 'L2', 'L0', 'L1', 'Y1', 'Y0', 'M2', '24', 'RU', '26', '27', '20', '21', '22', '23', '28', '29', 'RA', 'RO', 'RH', '2T', '2U', '2V', '2W', 'R0', '2Q', '2R', '2S', '2X', '2Y', '2Z', '2D', '2E', '2F', '2G', '2A', '2B', '2C', '2L', '2M', '2N', '2O', '2H', '2I', '2J', '2K', 'K1', '1B', '88', '89', '82', '83', '80', '81', '86', '87', '84', '85', 'E1', 'E0', 'E2

Next implement depth-first search (DFS). The function takes in a graph and a root node, and returns a list of the nodes visited during the search. The nodes should appear in the order in which they were *first visited*. The neighbors of a node should be processed in *alphabetical order*, where numbers come before letters. This will ensure that the output of your function is uniquely defined, given any input node.

In [6]:
def DFS(graph, root):
    """Runs depth-first search through a graph, starting at a given root. Neighboring
    nodes are processed in alphabetical order.
    
    Args:
        graph: the given graph, with nodes encoded as strings.
        root: the node from which to start the search.
        
    Returns:
        A list of nodes in the order in which they were first visited.
    """
    # TODO: implement function
    visited = []
    stack = [root,]
    while stack:
    # pop the component into stack
        node = stack.pop()
        if node not in visited:
    # Append the just visited node to the visited node
            visited.append(node)
            visitedSet = set(visited)
    # extend the stack with the new edges discovered
            stack.extend(reversed(sorted(list(set(graph.neighbors(node))-set(visited)))))
    return visited

We will check the correctness of your code by verifying that it correctly computes the DFS tree starting at Ulysses (node `OD`).

In [7]:
ulysses = DFS(G, 'OD')
# print ulysses

Next implement a function that finds the connected components of the character graph. When choosing roots for your components, always pick the *smallest unvisited node* according to alphabetical ordering. Combined with your DFS routine, this will ensure that the output is again uniquely defined.

In [8]:
def connected_components(graph):
    """Computes the connected components of the given graph.
    
    Args: 
        graph: the given graph, with nodes encoded as strings.
        
    Returns:
        The connected components of the graph. Components are listed in
        alphabetical order of their root nodes.
    """
    # TODO: implement function
    visited = []
    output = []
    while len(visited)-len(graph.nodes())!=0:
    # find the node to be explored next
        node = min(set(graph.nodes())-set(visited))
    # Keep track of what's already been visited
        visited.extend(list(DFS(graph,node))) 
    # Build the output
        output.append(list(DFS(graph,node)))
    return output
    

We will check correctness of your code by verifying that your output list is identical to our solution.

In [9]:
character_interactions = connected_components(G)
# print character_interactions

As a preliminary check, you should find that the following statements are all true.

In [10]:
component_sizes = [len(c) for c in character_interactions]
print "There are 12 connected components in the Iliad:", len(component_sizes) == 12
print "The giant component has size 542:", max(component_sizes) == 542
print "There are 5 isolated characters:", len([c for c in component_sizes if c == 1]) == 5

There are 12 connected components in the Iliad: True
The giant component has size 542: True
There are 5 isolated characters: True
